In [1]:
!pip install numpy==1.23.5 --quiet
!pip install pandas --upgrade --quiet
!pip install pandas_ta --upgrade --quiet

In [2]:
import pandas as pd
import numpy as np
import pandas_ta as ta

# Example usage
file_path = "/content/bitcoin_price_data(Jan-2015toMay2025).csv"
df = pd.read_csv(file_path)

In [3]:
df.head(5)

,Unnamed: 0,Date,Close,Open,High,Low,Volume,Daily_change
0,0,05/31/2025,"104,598.0","103,981.9","104,888.4","103,091.3",35.80K,0.59%
1,1,05/30/2025,"103,981.3","105,598.8","106,314.9","103,693.9",71.98K,-1.53%
2,2,05/29/2025,"105,600.0","107,759.1","108,896.7","105,399.9",70.02K,-2.03%
3,3,05/28/2025,"107,786.7","108,905.7","109,238.3","106,804.9",51.84K,-1.06%
4,4,05/27/2025,"108,944.0","109,455.3","110,718.7","107,572.2",65.02K,-0.47%


In [4]:
len(df)

3804

In [5]:
total_missing = df.isna().sum().sum()
print(f"Total missing values: {total_missing}")

Total missing values: 0


In [6]:
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

1. Unnamed: 0
2. Date
3. Close
4. Open
5. High
6. Low
7. Volume
8. Daily_change


In [7]:
df.drop(columns=['Unnamed: 0', 'Daily_change'], inplace=True)

In [8]:
df.head(5)

,Date,Close,Open,High,Low,Volume
0,05/31/2025,"104,598.0","103,981.9","104,888.4","103,091.3",35.80K
1,05/30/2025,"103,981.3","105,598.8","106,314.9","103,693.9",71.98K
2,05/29/2025,"105,600.0","107,759.1","108,896.7","105,399.9",70.02K
3,05/28/2025,"107,786.7","108,905.7","109,238.3","106,804.9",51.84K
4,05/27/2025,"108,944.0","109,455.3","110,718.7","107,572.2",65.02K


In [9]:
def convert_abbreviated_number(val):
    try:
        val = str(val).replace(',', '').strip()
        if val.endswith('K'):
            return float(val[:-1]) * 1_000
        elif val.endswith('M'):
            return float(val[:-1]) * 1_000_000
        elif val.endswith('B'):
            return float(val[:-1]) * 1_000_000_000
        else:
            return float(val)
    except:
        return np.nan  # or raise error if strict checking is desired

for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    df[col] = df[col].apply(convert_abbreviated_number)

In [10]:
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace(',', '').astype(float)

In [11]:
df.head(5)

,Date,Close,Open,High,Low,Volume
0,05/31/2025,104598.0,103981.9,104888.4,103091.3,35800.0
1,05/30/2025,103981.3,105598.8,106314.9,103693.9,71980.0
2,05/29/2025,105600.0,107759.1,108896.7,105399.9,70020.0
3,05/28/2025,107786.7,108905.7,109238.3,106804.9,51840.0
4,05/27/2025,108944.0,109455.3,110718.7,107572.2,65020.0


In [9]:
# # Remove commas and convert to float
# df['Close'] = df['Close'].str.replace(',', '').astype(float)

## **Indicators by Category**
Candles,
Cycles,
Momentum,
Overlap,
Performance,
Statistics,
Trend,
Utility,
Volatility,
Volume,

In [12]:
# List of Pandas TA categories.
df.ta.categories

['candles',
 'cycles',
 'momentum',
 'overlap',
 'performance',
 'statistics',
 'trend',
 'volatility',
 'volume']

In [13]:
df['Date'] = pd.to_datetime(df['Date'])        # Convert to datetime
df.set_index('Date', inplace=True)             # Set it as index
df.sort_index(inplace=True)                    # Sort chronologically

In [14]:
df.ta.datetime_ordered

True

## **Check if df['Close'] is cleaned and numeric**

In [15]:
print(df['Close'].dtype)

float64


In [16]:
print(df['Close'].isna().sum())

0


## **Statistical indicators (9)**

In [17]:
window = 20

df['entropy'] = ta.entropy(df['Close'], length=window)
df['kurtosis'] = ta.kurtosis(df['Close'], length=window)
df['mad'] = ta.mad(df['Close'], length=window)
df['median'] = ta.median(df['Close'], length=window)
df['quantile_0.5'] = ta.quantile(df['Close'], q=0.5, length=window)
df['skew'] = ta.skew(df['Close'], length=window)
df['stdev'] = ta.stdev(df['Close'], length=window)
df['variance'] = ta.variance(df['Close'], length=window)
df['zscore'] = ta.zscore(df['Close'], length=window)

## **Trend indicators (15)**

In [18]:
window = 20
fast = ta.ema(df['Close'], length=20)
slow = ta.ema(df['Close'], length=50)

df[['adx', 'dmp', 'dmn']] = ta.adx(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df[['amat_200_10', 'amat_signal']] = ta.amat(close=df['Close'])
df[['aroon_up', 'aroon_dn', 'aroon_osc']] = ta.aroon(high=df['High'], low=df['Low'], length=window)
df['chop'] = ta.chop(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df[['cksp_stop', 'cksp_direction']] = ta.cksp(high=df['High'], low=df['Low'], close=df['Close'])
df['decay'] = ta.decay(df['Close'], length=window)
df['decreasing'] = ta.decreasing(df['Close'], length=window)
df['dpo'] = ta.dpo(close=df['Close'], length=window, centered=False)
df['increasing'] = ta.increasing(df['Close'], length=window)
df['long_run'] = ta.long_run(fast=fast, slow=slow)

psar = ta.psar(high=df['High'], low=df['Low'], close=df['Close'])
psar.columns = ['psar_long', 'psar_short', 'psar_af', 'psar_reversal']
df = pd.concat([df, psar], axis=1)

df['qstick'] = ta.qstick(open_=df['Open'], close=df['Close'], length=window)
df['short_run'] = ta.short_run(fast=fast, slow=slow)
df['ttm_trend'] = ta.ttm_trend(high=df['High'], low=df['Low'], close=df['Close'])
df[['vortex_vi_plus', 'vortex_vi_minus']] = ta.vortex(high=df['High'], low=df['Low'], close=df['Close'], length=window)

/tmp/ipython-input-18-3488108943.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['decay'] = ta.decay(df['Close'], length=window)
/tmp/ipython-input-18-3488108943.py:10: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  df['decay'] = ta.decay(df['Close'], length=window)


## **Volatility (13)**

In [19]:
window = 20  # standard window for volatility indicators

aberration = ta.aberration(high=df['High'], low=df['Low'], close=df['Close'], length=window)
aberration.columns = ['aberration_upper','aberration_middle','aberration_lower','aberration_range']
df = pd.concat([df, aberration], axis=1)

bb = ta.bbands(close=df['Close'], length=window)
df[['bb_lower', 'bb_middle', 'bb_upper']] = bb[['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0']]

donchian = ta.donchian(high=df['High'], low=df['Low'], length=window)
donchian.columns = ['donchian_lower', 'donchian_middle', 'donchian_upper']
df = pd.concat([df, donchian], axis=1)

thermo = ta.thermo(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df[['thermo_raw', 'thermo_ema', 'thermo_long', 'thermo_short']] = thermo[['THERMO_20_2_0.5', 'THERMOma_20_2_0.5', 'THERMOl_20_2_0.5', 'THERMOs_20_2_0.5']]

df[['accbands_upper', 'accbands_middle', 'accbands_lower']] = ta.accbands(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df['atr'] = ta.atr(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df[['kc_upper', 'kc_middle', 'kc_lower']] = ta.kc(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df['massi'] = ta.massi(high=df['High'], low=df['Low'], length=window)
df['natr'] = ta.natr(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df['pdist'] = ta.pdist(open_=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])
df['rvi'] = ta.rvi(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df['true_range'] = ta.true_range(high=df['High'], low=df['Low'], close=df['Close'])
df['ui'] = ta.ui(close=df['Close'], length=window)

## **Performance (4)**

In [20]:
drawdown = ta.drawdown(close=df['Close'], cumulative=True)
df[['drawdown', 'drawdown_pct', 'drawdown_log']] = drawdown[['DD', 'DD_PCT', 'DD_LOG']]

trend = df['Close'] > ta.sma(df['Close'], length=50)
tr = ta.tsignals(trend=trend, close=df['Close'], asbool=True, cumulative=True)
df = pd.concat([df, tr], axis=1)

# Optional: rename for clarity
df.rename(columns={
    'TS_LOGR': 'trend_log_return',
    'TS_LOGR_C': 'trend_log_return_cum'
}, inplace=True)


df['log_return'] = ta.log_return(close=df['Close'], cumulative=True)
df['percent_return'] = ta.percent_return(close=df['Close'], cumulative=True)

## **Momentum (36)**

In [21]:
window = 20  # standard lookback period


df['ao'] = ta.ao(high=df['High'], low=df['Low'])
df['apo'] = ta.apo(close=df['Close'])
df['bias'] = ta.bias(close=df['Close'], length=window)
df['bop'] = ta.bop(open_=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])
df['cci'] = ta.cci(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df['cfo'] = ta.cfo(close=df['Close'], length=window)
df['cg'] = ta.cg(close=df['Close'], length=window)
df['cmo'] = ta.cmo(close=df['Close'], length=window)
df['coppock'] = ta.coppock(close=df['Close'])
df['er'] = ta.er(close=df['Close'], length=window)
df['inertia'] = ta.inertia(close=df['Close'], length=window)
df['mom'] = ta.mom(close=df['Close'], length=window)
df['pgo'] = ta.pgo(close=df['Close'], high=df['High'], low=df['Low'])
df['psl'] = ta.psl(close=df['Close'], length=window)
df['roc'] = ta.roc(close=df['Close'], length=window)
df['rsi'] = ta.rsi(close=df['Close'], length=window)
df['rsx'] = ta.rsx(close=df['Close'], length=window)
df['slope'] = ta.slope(close=df['Close'], length=window)
df['uo'] = ta.uo(high=df['High'], low=df['Low'], close=df['Close'])
df['willr'] = ta.willr(high=df['High'], low=df['Low'], close=df['Close'])


brar = ta.brar(open_=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])
df[['brar_br', 'brar_ar']] = brar[['AR_26', 'BR_26']]

eri = ta.eri(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df[['eri_bull', 'eri_bear']] = eri[['BULLP_20', 'BEARP_20']]

fisher = ta.fisher(high=df['High'], low=df['Low'], length=window)
df[['fisher', 'fisher_signal']] = fisher[['FISHERT_20_1', 'FISHERTs_20_1']]

kdj = ta.kdj(high=df['High'], low=df['Low'], close=df['Close'], length=window)
df[['kdj_k', 'kdj_d', 'kdj_j']] = kdj[['K_20_3', 'D_20_3', 'J_20_3']]

kst = ta.kst(close=df['Close'])
df[['kst', 'kst_signal']] = kst[['KST_10_15_20_30_10_10_10_15', 'KSTs_9']]

ppo = ta.ppo(close=df['Close'])
df[['ppo', 'ppo_signal', 'ppo_hist']] = ppo[['PPO_12_26_9', 'PPOs_12_26_9', 'PPOh_12_26_9']]

pvo = ta.pvo(volume=df['Volume'])
df[['pvo', 'pvo_signal', 'pvo_hist']] = pvo[['PVO_12_26_9', 'PVOs_12_26_9', 'PVOh_12_26_9']]

qqe = ta.qqe(close=df['Close'])
df[['qqe', 'qqe_signal']] = qqe[['QQE_14_5_4.236', 'QQEs_14_5_4.236']]

rvgi = ta.rvgi(open_=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], length=window)
df[['rvgi', 'rvgi_signal']] = rvgi[['RVGI_20_4', 'RVGIs_20_4']]

smi = ta.smi(close=df['Close'], length=window)
df[['smi', 'smi_signal', 'oscillator']] = smi[['SMI_5_20_5', 'SMIs_5_20_5', 'SMIo_5_20_5']]


squeeze = ta.squeeze(close=df['Close'], high=df['High'], low=df['Low'])
df = pd.concat([df, squeeze], axis=1)

stochrsi = ta.stochrsi(close=df['Close'])
df[['stochrsi_k', 'stochrsi_d']] = stochrsi[['STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3']]

trix = ta.trix(close=df['Close'])
df[['trix', 'trix_signal']] = trix[['TRIX_30_9', 'TRIXs_30_9']]

tsi = ta.tsi(close=df['Close'])
df[['tsi', 'tsi_signal']] = tsi[['TSI_13_25_13', 'TSIs_13_25_13']]

In [22]:
# Reset index to access datetime
df = df.reset_index()  # don't use drop=True

# Apply MACD
macd = ta.macd(close=df['Close'])
df[['macd', 'macd_signal', 'macd_hist']] = macd[['MACD_12_26_9', 'MACDs_12_26_9', 'MACDh_12_26_9']]

# Restore index using correct column name
df = df.set_index('Date')

In [23]:
df = df.reset_index()  # Resets 'Date' from index into a column
td_seq = ta.td_seq(close=df['Close'])
df = pd.concat([df, td_seq], axis=1)

# Restore index using 'Date', not 'index'
df = df.set_index('Date')

## **Candles (1)**

In [24]:
# Doji Candlestick
df['cdl_doji'] = ta.cdl_doji(open_=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])

## **Cycles (1)**

In [25]:
# Make a temporary reset-index DataFrame
df = df.reset_index()

# Apply ebsw safely
df['ebsw'] = ta.ebsw(close=df['Close'])

# Restore index
df = df.set_index('Date')

## **Volume (14)**

In [26]:
df['ad'] = ta.ad(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])
df['adosc'] = ta.adosc(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])
df['mfi'] = ta.mfi(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])
df['nvi'] = ta.nvi(close=df['Close'], volume=df['Volume'])
df['obv'] = ta.obv(close=df['Close'], volume=df['Volume'])
df['pvi'] = ta.pvi(close=df['Close'], volume=df['Volume'])
df['pvol'] = ta.pvol(close=df['Close'], volume=df['Volume'])
df['pvr'] = ta.pvr(close=df['Close'], volume=df['Volume'])
df['pvt'] = ta.pvt(close=df['Close'], volume=df['Volume'])
df['cmf'] = ta.cmf(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])
df['efi'] = ta.efi(close=df['Close'], volume=df['Volume'])
df['eom'] = ta.eom(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])


aobv = ta.aobv(close=df['Close'], volume=df['Volume'])
aobv.columns = ['aobv', 'aobv_min_2', 'aobv_max_2', 'aobv_ema_4', 'aobv_ema_12', 'aobv_lr_2', 'aobv_sr_2']
df = pd.concat([df, aobv], axis=1)

/tmp/ipython-input-26-3352240957.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.49572800e+07 6.13515533e+06 8.87487567e+06 ... 1.06928620e+10
 5.68419109e+09 5.25790637e+09]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['mfi'] = ta.mfi(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])
/tmp/ipython-input-26-3352240957.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[4.63847700e+06 1.40183620e+07 2.39826950e+07 ... 7.46638664e+09
 7.53366913e+09 3.73009389e+09]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['mfi'] = ta.mfi(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])


## **Overlap (31)**

In [27]:
df['alma'] = ta.alma(df['Close'])
df['dema'] = ta.dema(df['Close'])
df['ema'] = ta.ema(df['Close'])
df['fwma'] = ta.fwma(df['Close'])
df['hl2'] = ta.hl2(high=df['High'], low=df['Low'])
df['hlc3'] = ta.hlc3(high=df['High'], low=df['Low'], close=df['Close'])
df['hma'] = ta.hma(df['Close'])
df['hwma'] = ta.hwma(df['Close'])
df['kama'] = ta.kama(df['Close'])
df['linreg'] = ta.linreg(df['Close'])
df['midpoint'] = ta.midpoint(df['Close'])
df['midprice'] = ta.midprice(high=df['High'], low=df['Low'])
df['ohlc4'] = ta.ohlc4(open_=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])
df['pwma'] = ta.pwma(df['Close'])
df['rma'] = ta.rma(df['Close'])
df['sinwma'] = ta.sinwma(df['Close'])
df['sma'] = ta.sma(df['Close'])
df['ssf'] = ta.ssf(df['Close'])
supertrend_df = ta.supertrend(high=df['High'], low=df['Low'], close=df['Close'])
df = df.join(supertrend_df)
df['swma'] = ta.swma(df['Close'])
df['t3'] = ta.t3(df['Close'])
df['tema'] = ta.tema(df['Close'])
df['trima'] = ta.trima(df['Close'])
df['vidya'] = ta.vidya(df['Close'])
df['vwap'] = ta.vwap(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])
df['vwma'] = ta.vwma(close=df['Close'], volume=df['Volume'])
df['wcp'] = ta.wcp(high=df['High'], low=df['Low'], close=df['Close'])
df['wma'] = ta.wma(df['Close'])
df['zlma'] = ta.zlma(df['Close'])

/tmp/ipython-input-27-2903040892.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.957873027705299' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['vidya'] = ta.vidya(df['Close'])


In [28]:
hilo_df = ta.hilo(high=df['High'], low=df['Low'], close=df['Close'])
df = df.join(hilo_df)

In [29]:
ichimoku_a, ichimoku_b = ta.ichimoku(high=df['High'], low=df['Low'], close=df['Close'])
ichimoku_a = ichimoku_a.iloc[:, :-1]  # drop chikou span

# Keep only new columns not already in df
for col in ichimoku_a.columns:
    df[f"{col}_a"] = ichimoku_a[col]
for col in ichimoku_b.columns:
    df[f"{col}_b"] = ichimoku_b[col]


In [30]:
def mcginley_dynamic(close, k=10):
    mcgd = [close.iloc[0]]  # initial value
    for i in range(1, len(close)):
        prev = mcgd[-1]
        adj = (close.iloc[i] - prev) / (k * (close.iloc[i] / prev) ** 4)
        mcgd.append(prev + adj)
    return pd.Series(mcgd, index=close.index)

df['mcgd'] = mcginley_dynamic(df['Close'])

## **Data Processing**

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3804 entries, 2015-01-01 to 2025-05-31
Columns: 197 entries, Close to mcgd
dtypes: bool(3), float64(176), int64(18)
memory usage: 5.8 MB


In [32]:
df.dtypes.value_counts()

,count
float64,176
int64,18
bool,3


In [33]:
bool_cols = df.select_dtypes(include='bool').columns
print("Boolean columns:")
print(bool_cols.tolist())

Boolean columns:
['TS_Trends', 'TS_Entries', 'TS_Exits']


In [34]:
df[bool_cols].head()

,TS_Trends,TS_Entries,TS_Exits
Date,,,
2015-01-01,False,False,False
2015-01-02,False,False,False
2015-01-03,False,False,False
2015-01-04,False,False,False
2015-01-05,False,False,False


In [35]:
# Replace Boolean with 0/1 (for Crypto Signals)
# Convert True → 1 and False → 0

# Step 1: Identify boolean columns
bool_cols = ['TS_Trends', 'TS_Entries', 'TS_Exits']

# Step 2: Convert True/False to 1/0 (standard binary encoding)
df[bool_cols] = df[bool_cols].astype(int)

In [36]:
print(df[bool_cols].dtypes)
print(df[bool_cols].head())

TS_Trends     int64
TS_Entries    int64
TS_Exits      int64
dtype: object
            TS_Trends  TS_Entries  TS_Exits
Date                                       
2015-01-01          0           0         0
2015-01-02          0           0         0
2015-01-03          0           0         0
2015-01-04          0           0         0
2015-01-05          0           0         0


## **Post Processing**

In [39]:
missing_counts = df.isna().sum()
top_missing = missing_counts[missing_counts > 0].sort_values(ascending=False).head(15)
print(top_missing)

ISB_26_b         3804
ISA_9_b          3804
TD_SEQ_DNa       2090
psar_short       2060
SUPERTs_7_3.0    2040
qqe_signal       1934
SUPERTl_7_3.0    1771
psar_long        1745
TD_SEQ_UPa       1720
HILOs_13_21      1655
HILOl_13_21      1193
ISB_26_a           77
kst_signal         52
ISA_9_a            51
kst                44
dtype: int64


In [40]:
total_missing = df.isna().sum().sum()
print(f"Total missing values: {total_missing}")

Total missing values: 26519


In [41]:
len(df)

3804

In [42]:
df.drop(columns=['ISB_26_b', 'ISA_9_b', 'TD_SEQ_DNa', 'psar_short', 'SUPERTs_7_3.0',
 'qqe_signal', 'SUPERTl_7_3.0', 'psar_long', 'TD_SEQ_UPa', 'HILOs_13_21', 'HILOl_13_21'], inplace=True)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3804 entries, 2015-01-01 to 2025-05-31
Columns: 186 entries, Close to mcgd
dtypes: float64(165), int64(21)
memory usage: 5.6 MB


In [44]:
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

1. Close
2. Open
3. High
4. Low
5. Volume
6. entropy
7. kurtosis
8. mad
9. median
10. quantile_0.5
11. skew
12. stdev
13. variance
14. zscore
15. adx
16. dmp
17. dmn
18. amat_200_10
19. amat_signal
20. aroon_up
21. aroon_dn
22. aroon_osc
23. chop
24. cksp_stop
25. cksp_direction
26. decay
27. decreasing
28. dpo
29. increasing
30. long_run
31. psar_af
32. psar_reversal
33. qstick
34. short_run
35. ttm_trend
36. vortex_vi_plus
37. vortex_vi_minus
38. aberration_upper
39. aberration_middle
40. aberration_lower
41. aberration_range
42. bb_lower
43. bb_middle
44. bb_upper
45. donchian_lower
46. donchian_middle
47. donchian_upper
48. thermo_raw
49. thermo_ema
50. thermo_long
51. thermo_short
52. accbands_upper
53. accbands_middle
54. accbands_lower
55. atr
56. kc_upper
57. kc_middle
58. kc_lower
59. massi
60. natr
61. pdist
62. rvi
63. true_range
64. ui
65. drawdown
66. drawdown_pct
67. drawdown_log
68. TS_Trends
69. TS_Trades
70. TS_Entries
71. TS_Exits
72. log_return
73. percent_return
74.

In [45]:
feature_list = [
    'Close', 'Open', 'High', 'Low', 'Volume', 'entropy', 'kurtosis', 'mad', 'median', 'quantile_0.5',
    'skew', 'stdev', 'variance', 'zscore', 'adx', 'dmp', 'dmn', 'amat_200_10', 'amat_signal',
    'aroon_up', 'aroon_dn', 'aroon_osc', 'chop', 'cksp_stop', 'cksp_direction', 'decay', 'decreasing',
    'dpo', 'increasing', 'long_run', 'psar_af', 'psar_reversal', 'qstick', 'short_run', 'ttm_trend',
    'vortex_vi_plus', 'vortex_vi_minus', 'aberration_upper', 'aberration_middle', 'aberration_lower',
    'aberration_range', 'bb_lower', 'bb_middle', 'bb_upper', 'donchian_lower', 'donchian_middle',
    'donchian_upper', 'thermo_raw', 'thermo_ema', 'thermo_long', 'thermo_short', 'accbands_upper',
    'accbands_middle', 'accbands_lower', 'atr', 'kc_upper', 'kc_middle', 'kc_lower', 'massi', 'natr',
    'pdist', 'rvi', 'true_range', 'ui', 'drawdown', 'drawdown_pct', 'drawdown_log', 'TS_Trends',
    'TS_Trades', 'TS_Entries', 'TS_Exits', 'log_return', 'percent_return', 'ao', 'apo', 'bias', 'bop',
    'cci', 'cfo', 'cg', 'cmo', 'coppock', 'er', 'inertia', 'mom', 'pgo', 'psl', 'roc', 'rsi', 'rsx',
    'slope', 'uo', 'willr', 'brar_br', 'brar_ar', 'eri_bull', 'eri_bear', 'fisher', 'fisher_signal',
    'kdj_k', 'kdj_d', 'kdj_j', 'kst', 'kst_signal', 'ppo', 'ppo_signal', 'ppo_hist', 'pvo', 'pvo_signal',
    'pvo_hist', 'qqe', 'rvgi', 'rvgi_signal', 'smi', 'smi_signal', 'oscillator', 'SQZ_20_2.0_20_1.5',
    'SQZ_ON', 'SQZ_OFF', 'SQZ_NO', 'stochrsi_k', 'stochrsi_d', 'trix', 'trix_signal', 'tsi',
    'tsi_signal', 'macd', 'macd_signal', 'macd_hist', 'cdl_doji', 'ebsw', 'ad', 'adosc', 'mfi', 'nvi',
    'obv', 'pvi', 'pvol', 'pvr', 'pvt', 'cmf', 'efi', 'eom', 'aobv', 'aobv_min_2', 'aobv_max_2',
    'aobv_ema_4', 'aobv_ema_12', 'aobv_lr_2', 'aobv_sr_2', 'alma', 'dema', 'ema', 'fwma', 'hl2',
    'hlc3', 'hma', 'hwma', 'kama', 'linreg', 'midpoint', 'midprice', 'ohlc4', 'pwma', 'rma', 'sinwma',
    'sma', 'ssf', 'SUPERT_7_3.0', 'SUPERTd_7_3.0', 'swma', 't3', 'tema', 'trima', 'vidya', 'vwap',
    'vwma', 'wcp', 'wma', 'zlma', 'HILO_13_21', 'ISA_9_a', 'ISB_26_a', 'ITS_9_a', 'IKS_26_a', 'mcgd'
]


len(feature_list)

186

In [46]:
# Save the final DataFrame with the 'target' column
df.to_csv("crypto_signals.csv", index=True)  # index=True to keep the Date index